## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Возможные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd


data = pd.read_csv('/content/drive/MyDrive/ITMO/5-sem/ml/women-clothing-accessories.csv', sep='\t')
data.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [ ]:
data['sentiment'].unique()

array(['negative', 'neautral', 'positive'], dtype=object)

In [ ]:
data['sentiment'].value_counts()

negative    30000
neautral    30000
positive    30000
Name: sentiment, dtype: int64

In [ ]:
data['review_lower'] = data['review'].str.lower()

In [ ]:
data['review_replaced_mcl'] = data['review_lower'].str.replace('ъ', 'ь').replace('й', 'и').replace('ё', 'е')

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

def tokenize_text(text):
    return word_tokenize(text)

data['review_tokenized'] = data['review_replaced_mcl'].apply(tokenize_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from string import punctuation
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('russian')

def remove_stop_words_and_punctuation(text):
    filtered_tokens = [word for word in text if ((word not in stop_words )\
                                                 and (word not in punctuation))]
    return filtered_tokens

data['review_without_stop_and_punct'] = data['review_tokenized'].apply(remove_stop_words_and_punctuation)
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review,sentiment,review_lower,review_replaced_mcl,review_tokenized,review_without_stop_and_punct
0,качество плохое пошив ужасный (горловина напер...,negative,качество плохое пошив ужасный (горловина напер...,качество плохое пошив ужасный (горловина напер...,"[качество, плохое, пошив, ужасный, (, горловин...","[качество, плохое, пошив, ужасный, горловина, ..."
1,"Товар отдали другому человеку, я не получила п...",negative,"товар отдали другому человеку, я не получила п...","товар отдали другому человеку, я не получила п...","[товар, отдали, другому, человеку, ,, я, не, п...","[товар, отдали, другому, человеку, получила, п..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"ужасная синтетика! тонкая, ничего общего с пре...","ужасная синтетика! тонкая, ничего общего с пре...","[ужасная, синтетика, !, тонкая, ,, ничего, общ...","[ужасная, синтетика, тонкая, общего, представл..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"товар не пришел, продавец продлил защиту без м...","товар не пришел, продавец продлил защиту без м...","[товар, не, пришел, ,, продавец, продлил, защи...","[товар, пришел, продавец, продлил, защиту, мое..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"кофточка голая синтетика, носить не возможно.","кофточка голая синтетика, носить не возможно.","[кофточка, голая, синтетика, ,, носить, не, во...","[кофточка, голая, синтетика, носить, возможно]"


In [ ]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

def lemmatize(tokens):
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

data['review_lemmitized'] = data['review_without_stop_and_punct'].apply(lemmatize)
data.head()

,review,sentiment,review_lower,review_replaced_mcl,review_tokenized,review_without_stop_and_punct,review_lemmitized
0,качество плохое пошив ужасный (горловина напер...,negative,качество плохое пошив ужасный (горловина напер...,качество плохое пошив ужасный (горловина напер...,"[качество, плохое, пошив, ужасный, (, горловин...","[качество, плохое, пошив, ужасный, горловина, ...","[качество, плохой, пошив, ужасный, горловина, ..."
1,"Товар отдали другому человеку, я не получила п...",negative,"товар отдали другому человеку, я не получила п...","товар отдали другому человеку, я не получила п...","[товар, отдали, другому, человеку, ,, я, не, п...","[товар, отдали, другому, человеку, получила, п...","[товар, отдать, другой, человек, получить, пос..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"ужасная синтетика! тонкая, ничего общего с пре...","ужасная синтетика! тонкая, ничего общего с пре...","[ужасная, синтетика, !, тонкая, ,, ничего, общ...","[ужасная, синтетика, тонкая, общего, представл...","[ужасный, синтетик, тонкий, общий, представить..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"товар не пришел, продавец продлил защиту без м...","товар не пришел, продавец продлил защиту без м...","[товар, не, пришел, ,, продавец, продлил, защи...","[товар, пришел, продавец, продлил, защиту, мое...","[товар, прийти, продавец, продлить, защита, мо..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"кофточка голая синтетика, носить не возможно.","кофточка голая синтетика, носить не возможно.","[кофточка, голая, синтетика, ,, носить, не, во...","[кофточка, голая, синтетика, носить, возможно]","[кофточка, голый, синтетик, носить, возможно]"


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data.review_lemmitized, data.sentiment, train_size = 0.7)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

# сетка параметров
parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__max_df': [0.9, 0.95],
    'tfidf__min_df': [0.1, 0.05],
    'classifier__alpha': [0.1, 0.5, 1.0]
}

grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)
%time grid_search.fit(x_train.apply(lambda x: ' '.join(x)), y_train)

print("Best parameters found: ", grid_search.best_params_)

y_pred = grid_search.predict(x_test.apply(lambda x: ' '.join(x)))

print(classification_report(y_test, y_pred))
"""              precision    recall  f1-score   support

    neautral       0.46      0.60      0.52      8939
    negative       0.67      0.53      0.59      9079
    positive       0.65      0.61      0.63      8983

    accuracy                           0.58     27001
   macro avg       0.60      0.58      0.58     27001
weighted avg       0.60      0.58      0.58     27001
"""

CPU times: user 9.06 s, sys: 3.94 s, total: 13 s
Wall time: 4min 15s
Best parameters found:  {'classifier__alpha': 0.1, 'tfidf__max_df': 0.9, 'tfidf__min_df': 0.05, 'tfidf__ngram_range': (1, 1)}
              precision    recall  f1-score   support

    neautral       0.46      0.60      0.52      8939
    negative       0.67      0.53      0.59      9079
    positive       0.65      0.61      0.63      8983

    accuracy                           0.58     27001
   macro avg       0.60      0.58      0.58     27001
weighted avg       0.60      0.58      0.58     27001



'              precision    recall  f1-score   support\n\n    neautral       0.46      0.60      0.52      8939\n    negative       0.67      0.53      0.59      9079\n    positive       0.65      0.61      0.63      8983\n\n    accuracy                           0.58     27001\n   macro avg       0.60      0.58      0.58     27001\nweighted avg       0.60      0.58      0.58     27001\n'

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах.

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb')
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**:
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?
Из-за пересечения

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
sentence = "Привет Друг, как твои дела, а?"
result = re.findall(r'\b\w{2}', sentence)
result

['Пр', 'Др', 'ка', 'тв', 'де']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
sentence = "Привет. Как твои дела? У меня тоже хорошо. Да."
result = re.split(r'[.!?]', sentence, maxsplit=2)
result

['Привет', ' Как твои дела', ' У меня тоже хорошо. Да.']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
original_string = "76 лю6лю с80ю маму на 100%."
result = re.sub(r'\d', 'DIG', original_string)
result

'DIGDIG люDIGлю сDIGDIGю маму на DIGDIGDIG%.'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
url_regexp = r"((www\.[^\s]+)|(https?://[^\s]+))"
input_text = "Привет.Вот ссылка: https://www.example.com, www.google.com/search"

result = re.sub(url_regexp, '', input_text)
# result = re.sub(url_http, '', input_text)
# result = re.sub(url, '', result)
result

'Привет.Вот ссылка:  '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
input_string = "Слова? Да, больше, ещё больше слов! Что-то ещё."
regex_pattern = re.compile('[А-Яа-яё\-]{4,}')
result = regex_pattern.findall(input_string)
result

['Слова', 'больше', 'больше', 'слов', 'Что-то']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
email_addresses = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"

regex_pattern = re.compile(r'@[a-zA-Z0-9.-]+')

result = regex_pattern.findall(email_addresses)
result

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']